In [1]:
from haystack import Pipeline
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument  # for PDFs
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
import os
from pathlib import Path


In [2]:
# 1. Setup environment
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

# 2. Initialize DocumentStore
document_store = InMemoryDocumentStore()

# 3. Convert your PDF(s) or text files into Document objects
converter = PyPDFToDocument()  # for PDFs; for text files you may skip
docs = converter.run(sources=[Path("info.pdf")])

In [3]:
docs["documents"]

[Document(id=12977907e535797c23d1e7b24b702e4852bae79642e42a38234b427aac6f03df, content: 'Miles’s favorite color is Red. 
 Lucy’s favorite color is Green. 
 Miles’s dog’s name is Firu. He is a...', meta: {'file_path': 'info.pdf'})]

In [4]:
# 4. Embed documents
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()
emb_docs = doc_embedder.run(docs["documents"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# 5. Write to store
document_store.write_documents(emb_docs["documents"])


1

In [6]:

# 6. Setup retriever + query embedder
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

# 7. Build the RAG pipeline
template = [
    ChatMessage.from_user(
        """
        Given the following context, answer the question.

        {% for doc in documents %}
        {{ doc.content }}
        {% endfor %}

        Question: {{ question }}
        Answer:
        """
    )
]
prompt_builder = ChatPromptBuilder(template=template)
chat_generator = OpenAIChatGenerator(model="gpt-5-nano")  # or another LLM

pipeline = Pipeline()
pipeline.add_component("text_embedder", text_embedder)
pipeline.add_component("retriever", retriever)
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("llm", chat_generator)

pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
pipeline.connect("retriever", "prompt_builder")
pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [7]:
# 8. Ask a question
question = "What is Miles' favorit color?"
result = pipeline.run({
    "text_embedder": {"text": question},
    "prompt_builder": {"question": question}
})
print(result["llm"]["replies"][0].text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Red.


In [8]:
question = "Tell me about Firu"
result = pipeline.run({
    "text_embedder": {"text": question},
    "prompt_builder": {"question": question}
})
print(result["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Firu is Miles’s dog. He is a Cavalier King Charles Spaniel / Papillon mix.
